In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostRegressor
from tqdm import tqdm
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from scripts.datasets import create_folds
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
#model = XGBRegressor()
#model = AdaBoostRegressor()

In [33]:
X, y, groups, group_kfold = create_folds()
error = 0
total = 0
for fold_idx, (train_idx, test_idx) in enumerate(group_kfold.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    X_train = X_train.drop(columns=["Year", "Player", "Team"])
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    model.fit(X_train, y_train)
    X_test_clean = X_test.drop(columns=["Year", "Player", "Team"])
    X_test_clean = scaler.transform(X_test_clean)
    y_preds = model.predict(X_test_clean)
    X_test2 = X_test.copy()
    X_test2["Predicted"] = y_preds
    X_test2["Actual"] = y_test
    X_predict = X_test2.sort_values("Predicted", ascending=False)
    X_real = X_test2.sort_values("Actual", ascending=False)
    total += 1
    print(X_test["Year"].iloc[0])
    print(X_real["Player"].iloc[0], X_predict["Player"].iloc[0])
    print(X_real["Player"].iloc[1], X_predict["Player"].iloc[1])
    print(X_real["Player"].iloc[2], X_predict["Player"].iloc[2])
    if X_real["Player"].iloc[0] != X_predict["Player"].iloc[0]:
        error += 1
    print(mean_squared_error(y_test, y_preds), r2_score(y_test, y_preds))
        
print(f"Accuracy: {(total - error) / total}")

2024
Nikola Jokić Nikola Jokić
Shai Gilgeous-Alexander Shai Gilgeous-Alexander
Luka Dončić Giannis Antetokounmpo
0.14221119506104163 0.8577888049389584
2023
Joel Embiid Nikola Jokić
Nikola Jokić Giannis Antetokounmpo
Giannis Antetokounmpo Joel Embiid
0.17583098705939745 0.8241690129406025
1997
Karl Malone Michael Jordan
Michael Jordan Karl Malone
Grant Hill Grant Hill
0.08453006941734395 0.915469930582656
2016
Stephen Curry Stephen Curry
Kawhi Leonard Kevin Durant
LeBron James LeBron James
0.23523496721480838 0.7647650327851915
2013
LeBron James LeBron James
Kevin Durant Kevin Durant
Carmelo Anthony Russell Westbrook
0.1436899342835113 0.8563100657164888
2008
Kobe Bryant LeBron James
Chris Paul Chris Paul
Kevin Garnett Kobe Bryant
0.4617049357034234 0.5382950642965767
2012
LeBron James LeBron James
Kevin Durant Kevin Durant
Chris Paul Chris Paul
0.16166771752853523 0.8383322824714647
2017
Russell Westbrook James Harden
James Harden Kawhi Leonard
Kawhi Leonard Russell Westbrook
0.258685